# Détection de Fake News

L'objectif est de prédire si des déclarations sont vraies ou fausses.

Les données sont celles de la base LIAR2

- accessible depuis Hugging Face: https://huggingface.co/datasets/chengxuphd/liar2 
- ou depuis le Github https://github.com/chengxuphd/liar2 qui contient des codes et des références.

## Les données


### Chargement des données

Depuis Hugging-Face

Les données sont organisées en dictionnaires.

In [1]:
import numpy as np
import datasets
dataset = "chengxuphd/liar2"
dataset = datasets.load_dataset(dataset)
statement_train, y_train = dataset["train"]["statement"], dataset["train"]["label"]
statement_val, y_val = dataset["validation"]["statement"], dataset["validation"]["label"]
statement_test, y_test = dataset["test"]["statement"], dataset["test"]["label"]

output_tags = ["Pants on fire","False","Barely-true","Half-true","Mostly-true","True"]

Depuis le Github de LIAR2

Les données sont téléchargées en fichier 'csv' et lus avec pandas.

In [2]:
import os
import pandas as pd

if (not os.path.exists('train.csv')):
  !wget https://raw.githubusercontent.com/chengxuphd/liar2/refs/heads/main/liar2/train.csv
if (not os.path.exists('test.csv')):
  !wget https://raw.githubusercontent.com/chengxuphd/liar2/refs/heads/main/liar2/test.csv

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')


--2025-12-23 16:23:28--  https://raw.githubusercontent.com/chengxuphd/liar2/refs/heads/main/liar2/train.csv
Resolving proxy.onera (proxy.onera)... 125.1.1.78
Connecting to proxy.onera (proxy.onera)|125.1.1.78|:80... connected.
Proxy request sent, awaiting response... 200 OK
Length: 19041186 (18M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  18.16M  56.5MB/s    in 0.3s    

2025-12-23 16:23:29 (56.5 MB/s) - ‘train.csv’ saved [19041186/19041186]

--2025-12-23 16:23:29--  https://raw.githubusercontent.com/chengxuphd/liar2/refs/heads/main/liar2/test.csv
Resolving proxy.onera (proxy.onera)... 125.1.1.78
Connecting to proxy.onera (proxy.onera)|125.1.1.78|:80... connected.
Proxy request sent, awaiting response... 200 OK
Length: 2384586 (2.3M) [text/plain]
Saving to: ‘test.csv’

test.csv            100%[===================>]   2.27M  --.-KB/s    in 0.1s    

2025-12-23 16:23:30 (21.5 MB/s) - ‘test.csv’ saved [2384586/2384586]



### Nettoyage des données

Les fonctions ci-dessous transforment les données brutes en une séquence de mots normalisés.

In [4]:
import re

import nltk
# Interactive library download
#nltk.download()
nltk.download('punkt_tab')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# ensure NLTK resources available
nltk_packages = ["punkt", "stopwords", "wordnet", "omw-1.4"]
for pkg in nltk_packages:
    try:
        nltk.data.find(pkg)
    except LookupError:
        nltk.download(pkg)


STOPWORDS = set(stopwords.words("english"))
LEMMATIZER = WordNetLemmatizer()

def clean_text(text: str) -> str:
    """Basic cleanup: lowercase, remove urls, punctuation, extra spaces, stopwords, lemmatize."""
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", " ", text)
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    tokens = nltk.word_tokenize(text)
    tokens = [t for t in tokens if t not in STOPWORDS and len(t) > 1]
    tokens = [LEMMATIZER.lemmatize(t) for t in tokens]
    return " ".join(tokens)

statement_train_clean = list(map(clean_text, statement_train))
statement_val_clean = list(map(clean_text, statement_val))
statement_test_clean = list(map(clean_text, statement_test))

[nltk_data] Downloading package punkt_tab to /d/herbin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /d/herbin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /d/herbin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /d/herbin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /d/herbin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Génération des labels

Les données d'origine sont définies par un score de véracité à 6 niveaux.

Ici on transforme le problème en une classification binaire ("true", "Fake").

In [5]:
y_test_binary = np.array([0 if i <= 2 else 1 for i in y_test])
y_train_binary = np.array([0 if i <= 2 else 1 for i in y_train])
y_val_binary = np.array([0 if i <= 2 else 1 for i in y_val])

## Premiers tests

### Transformation des données

Les déclarations sont des listes de taille variable.

On les transforme en vecteurs de taille fixe par la méthode des "bag of words", dans sa version Tf-Idf.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train = vec.fit_transform(statement_train_clean)
X_test = vec.transform(statement_test_clean)

print("Les données sont de dimension {:d}".format(X_train.shape[1]))
print("Le nombre de données d'apprentissage est {:d}".format(y_train_binary.shape[0]))

# Calcul du nombre moyens d'éléments non nuls
non_nul_moyen = np.mean(np.sum(X_train > 0,axis=1))
print("Le nombre moyen de valeurs non nulles par échantillon est {:.1f}".format(non_nul_moyen))

Les données sont de dimension 10000
Le nombre de données d'apprentissage est 18369
Le nombre moyen de valeurs non nulles par échantillon est 12.2


### Essais élémentaire de classification

Le code ci-dessous teste différents algorithmes du cours pour la classification binaire, avec leurs paramètres par défaut.

C'est une première base, clairement à améliorer.

En particulier, on peut utiliser un modèle de langage pour encoder les déclarations et travailler sur cette représentatino plutôt que sur le "bag of words".

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

model_list = [MultinomialNB(), 
              LinearSVC(max_iter=10000, C=0.1), 
              LogisticRegression(max_iter=1000), 
            #   RandomForestClassifier(n_estimators=200, n_jobs=-1)
              ]

for model in model_list:
    model.fit(X_train, y_train_binary)
    preds = model.predict(X_train)

    print("Erreurs de train/test pour le modèle {} est {:.2f} / {:.2f}%".format(
        model, 
        100*(1-model.score(X_train, y_train_binary)), 
        100*(1-model.score(X_test, y_test_binary))))

Erreurs de train/test pour le modèle MultinomialNB() est 23.21 / 32.40%
Erreurs de train/test pour le modèle LinearSVC(C=0.1, max_iter=10000) est 21.68 / 31.53%
Erreurs de train/test pour le modèle LogisticRegression(max_iter=1000) est 21.37 / 31.66%


### Autres informations utilisables

Le jeu de données contient en plus de la déclaration ("statement") et de sa valeur de vérité ("label") des informations sur la data, le sujet, l'orateur, sa description, le lieu, le contexte de la déclaration, et une justification du score. Il contient aussi un historique pour chaque orateur de la véracité de ses déclarations ('counts').

Elle peuvent être utilisées pour améliorer la prédiction en exploitant les bons priors...

In [8]:

'''
print(dataset['train'])
for i in range(4):
    print('statement: ', dataset['train']['statement'][i])
    print('speaker: ', dataset['train']['speaker'][i])
    print('speaker description: ', dataset['train']['speaker_description'][i])
    print('context: ', dataset['train']['context'][i])
    print('justification: ', dataset['train']['justification'][i])
    print('')
'''

df_train.head()


,id,label,statement,date,subject,speaker,speaker_description,state_info,true_counts,mostly_true_counts,half_true_counts,mostly_false_counts,false_counts,pants_on_fire_counts,context,justification
0,13847,5,"90 percent of Americans ""support universal bac...","October 2, 2017",government regulation;polls and public opinion...,chris abele,"Chris Abele is Milwaukee County Executive, a p...",wisconsin,1,4,5,3,5,2,a tweet,"""Universal"" is the term for background checks ..."
1,13411,1,Last year was one of the deadliest years ever ...,"May 19, 2017",after the fact;congress;criminal justice;histo...,thom tillis,Thom Tillis is a Republican who serves as U.S....,north carolina,0,2,7,3,2,0,a press release supporting the Back The Blue A...,"Sen. Thom Tillis, a North Carolina Republican,..."
2,10882,0,"Bernie Sanders's plan is ""to raise your taxes ...","October 28, 2015",taxes,chris christie,"Chris Christie announced June 6, 2023 that he ...",national,21,20,27,11,17,8,"Boulder, Colo","Christie said that Sanders’s plan is ""to raise..."
3,20697,4,Voter ID is supported by an overwhelming major...,"December 8, 2021",voter id laws,lee zeldin,Lee Zeldin is a Republican representing New Yo...,new york,1,2,0,0,0,0,a Tweet,Zeldin claimed voter identification requiremen...
4,6095,2,"Says Barack Obama ""robbed Medicare (of) $716 b...","August 12, 2012",federal budget;history;medicare;retirement,mitt romney,Mitt Romney is a U.S. senator from Utah. He ra...,national,31,33,58,35,32,19,"an interview on ""60 Minutes""","Romney said, ""There's only one president that ..."
